# Либы

In [214]:
import pandas as pd
import numpy as np

In [192]:
import geocoder

In [219]:
from scipy.stats import kstest, norm

In [229]:
import string

# Подгрузка данных

In [193]:
data = pd.read_csv('data/dataset.tskv', sep='\t')
data.head()

,"address=Екатеринбург, ул. Московская / ул. Волгоградская / ул. Печатников",name_ru=Московский квартал,rating=3.,rubrics=Жилой комплекс,"text=Московский квартал 2.\nШумно : летом по ночам дикие гонки. Грязно : кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. Детские площадки убогие, на большой площади однотипные конструкции. Очень дорогая коммуналка. Часто срабатывает пожарная сигнализация. Жильцы уже не реагируют. В это время, обычно около часа, не работают лифты. Из плюсов - отличная планировка квартир ( Московская 194 ), на мой взгляд. Ремонт от застройщика на 3-. Окна вообще жуть - вместо вентиляции. По соотношению цена/качество - 3."
0,"address=Московская область, Электросталь, прос...",name_ru=Продукты Ермолино,rating=5.,rubrics=Магазин продуктов;Продукты глубокой за...,"text=Замечательная сеть магазинов в общем, хор..."
1,"address=Краснодар, Прикубанский внутригородско...",name_ru=LimeFit,rating=1.,rubrics=Фитнес-клуб,"text=Не знаю смутят ли кого-то данные правила,..."
2,"address=Санкт-Петербург, проспект Энгельса, 11...",name_ru=Snow-Express,rating=4.,rubrics=Пункт проката;Прокат велосипедов;Сапсё...,text=Хорошие условия аренды. \nДружелюбный пер...
3,"address=Тверь, Волоколамский проспект, 39",name_ru=Студия Beauty Brow,rating=5.,"rubrics=Салон красоты;Визажисты, стилисты;Сало...",text=Топ мастер Ангелина топ во всех смыслах )...
4,"address=Иркутская область, Черемхово, Первомай...",name_ru=Tele2,rating=5.,rubrics=Оператор сотовой связи;Интернет-провайдер,"text=Приятное общение, все доступно объяснили,..."


## обработка нанов

In [194]:
data.isna().sum().sum() / data.shape[0] * 100

0.19400038800077601

просто удалим пропуски, их доля чрезвычайно мала в общей массе (0.19 %)

In [195]:
data = data.dropna()

# Приводим в читаемое состояние

In [196]:
def columns_names_preprop(x):
    sym_num = x.find('=')
    return x[:sym_num + 1][:-1]

In [197]:
def data_preprop(x):
    return x.map(lambda y: y[y.find('=') + 1:])

In [198]:
data.columns = list(map(columns_names_preprop, data.columns))
data = data.apply(lambda x: data_preprop(x))
    
data['rating'] = data['rating'].apply(lambda x: int(x[:-1]))

# Анализ данных

## обработка дубликатов

In [199]:
data.duplicated(subset=['address', 'name_ru', 'rating', 'rubrics']).sum()

175939

In [200]:
data[(data['address'] == 'Ростов-на-Дону, улица Мечникова, 150')].iloc[0, 4]

'Сдавал комп в чистку, всё сделали за один день. Сам комп стал работать чуть плавнее, а главное, стал намного тише. Дружелюбные и профессиональные ребята, буду обращаться ещё!'

In [201]:
data[(data['address'] == 'Ростов-на-Дону, улица Мечникова, 150')].iloc[1, 4]

'Отличный сервисный центр и магазин ПК, приятный и вежливый персонал, доступные и оправданные цены, всем советую, не пожалеете '

я уже хотел удалять 175_939 дубликатов, а датасет оказвается про комментарии

In [202]:
data.duplicated().sum()

1

In [203]:
data = data.drop_duplicates()

In [ ]:
data.to_csv('data/filtered.csv', index=False)

# Препроцессинг

In [259]:
pos_words = {'счастливый', 'нравится', 'люблю', 'хороший', 'любимый', 'лучший', 'отличный', 'удивительный', 'радостный', 'позитивный'}
neg_words = {'грустный', 'не нравится', 'ненавижу', 'злой', 'ужасный', 'плохой', 'худший', 'скучный', 'отрицательный'}

оставляем в rubrics только первое значение, чтобы избежать перечислений

In [260]:
def count_words(x, lst):
    for p in string.punctuation:
        if p in x:
                # банальная замена символа в строке
            x = x.replace(p, '')
                
    x = x.lower()

    x_set = set(x.split(' '))

    return len(x_set & lst)

In [261]:
def preprocess(data, is_train=True):
    data['rubrics'] = data['rubrics'].apply(lambda x: x.split(';')[0])

    data['region'] = data['address'].apply(lambda x: x.split(',')[0])
    
    data['num_bad_words'] = data['text'].apply(lambda x: count_words(x, neg_words))
    data['num_good_words'] = data['text'].apply(lambda x: count_words(x, pos_words))

    return data

In [262]:
data = preprocess(data)

# Смотрим корреляции

In [220]:
kstest(data['rating'], lambda x: norm.cdf(x, loc=5, scale=10))

KstestResult(statistic=0.5, pvalue=0.0, statistic_location=5, statistic_sign=1)

распределние нормальное

смотрим корреляцию eta для признаков категориальных rubrics и region и количественного rating

In [222]:
num_col = 'rating'

cat_cols = ['rubrics', 'region']

for cat_col in cat_cols:
    groups = [data[num_col][data[cat_col] == value] for value in data[cat_col].unique()]
            
    total_variance = np.var(data[num_col], ddof=1)
    group_means = [g.mean() for g in groups]
    group_sizes = [len(g) for g in groups]
    group_variance = sum(size * (mean - data[num_col].mean()) ** 2 for size, mean in zip(group_sizes, group_means)) / len(data)

    print(group_variance / total_variance)

0.046712209629759444
0.002863443327267895


0.046712209629759444

0.002863443327267895

регион имеет очень незначительную корреляцию с рейтингом, у рубрики она несколько больше


In [225]:
data.columns

Index(['address', 'name_ru', 'rating', 'rubrics', 'text', 'region'], dtype='object')

In [ ]:
data.corr(numeric_only=True)

# Описание атрибутов

address - полный адрес заведения

name_ru - официальное название владельца

ratig - рейтинг заведения

rubrics - основной вид деятельности заведения

text - комментарий

region - регион или город заведения